In [1]:
import requests
import datetime
# import h3
import networkx as nx
import urllib
import numpy as np
import pandas as pd
import time
import random
from tqdm import tqdm

def list_hotspots_in_city(city_id: str):
    """retrieves all the hotspots in a city"""
    url = 'https://api.helium.io/v1/cities/' + city_id + '/hotspots'
    r = requests.get(url)
    hotspots = r.json()['data']
    return hotspots


def list_witnesses_for_hotspot(hotspot_address: str):
    """lists a hotspot's witnesses over the last 5 days"""
    url = 'https://api.helium.io/v1/hotspots/' + hotspot_address + '/witnesses'
    r = requests.get(url)
    witnesses = r.json()['data']
    return witnesses


def get_hotspot_rewards(hotspot_address: str, n_days: int):
    """Get a hotspot's cumulative rewards for the past n days"""
    start_time = datetime.datetime.isoformat(datetime.datetime.now() - datetime.timedelta(days=n_days))
    url = f"https://api.helium.io/v1/hotspots/{hotspot_address}/rewards/sum?min_time={start_time}"
    r = requests.get(url)
    rewards = r.json()['data']['total']
    return rewards


def get_hotspot_details(address: str):
    url = 'https://api.helium.io/v1/hotspots/' + address
    r = requests.get(url)
    details = r.json()['data']
    return details

Additional questions:     

    1. What is significance, in any, of when one device asserts a location in a hex relative to others who come before or later?     
    2. To what extent is there a randomness factor built into the blockchain?     
    3. lf there is a randomness factor, is it fixed or does it change continuously over certain periods of time?

### Addresses for active Helium Billboards Partners hotspots

In [2]:
hbp_ids_list = [
'11r9njKS6SJc1czPkagCq1wpZngPj2knKuzNuKqFbCVR5sR1XaJ',
'11ghRVmA1GbqTJjpUQUPBxFnXnXk7BVcECPLogWtVKJzYqrv6MY',
'112jtbdZbakFizzwRq3GLXfEVkzokUU8LYX2v6M1pEYEiGfv1vYR',
'11sbc4LMmdoMKNDdikcyG8sYJgoNhQiyFJiQMMb5HZ2VVZLjhpe',
'112rUYDqYRLP1rLdb7Qh9LhLjmppaVXpnqqJugBWqGKcgJYgC9vB',
'112m8GNtuhcpBYVpfmmh7pyPhuGqKBAwkjs6hVs6w37RXLt5TApa',
'116pywYRxMZtWYSaQ8E5j4zfeJtNw9tiW38UohxpB6iD9kRGrwW',
'11167evrDdLmQW1BhcnAKw14UyWrGJtNnvTVEG8ndgkWHFDFJx7',
'112jvS4rm5neVt3oTyUm6dDeCp3MCw5DukoGUXVbEH28kXhwM4rB',
'112dyM1SBEmFBPNB68t5vzwcbkYFSWGdSf7wKtwdC4vvUaivHbP9',
'11TKTiWqSTvs4BxKJfbhayVrGBkbbMtmoivfZj4c7PrfQzMobAE',
'112TESQEF1p4Kp6YQkn2n3RJppENKx7cPZcgAKZ51WgmszoZCqDW',
'11E2Pi9ewv1fn4ZFQLxv33eHHRpLreA3WWQ3srSSamWaZsccZZD',
'112tLoa3VwfM26BVyDijAaGbzWiQEbQsbcRFG9C4sgm2LG8LR18e',
'11EgNWzxBEMGXBxShHaBUz5F6krNoV25v3oVJDRh7E1xzdvva8w',
'112vYWwbP8adKs5vy4xoagQp6tV5CdmZB31f6MtKtSv1gsjZ2Nwm',
'112auKBDzTaweoCsCvceeMR94DMuMaiLrHwhdGaLSnYvWyzefiZF',
'112w7W3q6Nfj8945KtpyKYbN3E28g6xt99NHDnqb9QpcXNnTWSW5',
'1129zmWmeZanXdZbCZNtPhUCnG3MLKpeZnpwCNewxEKjRU7c6SrX',
'11krwFPSkFWEk3bdmUDoKWp8KgGAREkEvq7CZvdoMx1SNHAuMKX',
'112Auve4H3myt61VwzjWQv3ARwqzMC2tuEy9MH9CvuaKy3ktPVbj',
'112edBj8jtw2LywnBfR8iUVDXV6JrM8wHp8mdCNd6DzdP7temAua',
'112hGNPapSrj7FCcJ86mYVjpzSMXmv6Tu6cu22JCYGFReCBsqPRE',
'11X19byPQ4He3bgpEU8e6ZFVYtzJ4xM4MUwp6BjZ54C5iu1Cqqp',
'11x3cvzDYG96t7TWr2WxMGunrzxcczrvMD2SbAJ5u9c5VNivDyK',
'118vc66eAvMBHdtyKKpmqaq1Ga2Wf3qdjPuK9VqH8nQyjKeDsXM',
'11bjAvVq5ZifLG19jrJGav2KDgpGkttaKG8YkcsdHm1csgJbeaD',
'1127kP6J4zGGkvJsCAh9oCFskRNJBwjUvyMKVGBP4672znbVWDaf',
'112U9ohNQbG2cStjVnVXnmrk4f2ZAEpBN3EA5CaLMbicPCAcEHVN',
'112VKAZh4ydMZxLvpYmLqr4KdbznPdMrsi4ga2qUP9q5K8cB2QgD',
'112cF4LdRUip55Ty5V7igxGdoNphCPqkAytoYtBevrP4SWmUhMgX',
'11W8BriDRakjyvot9DoefCUWgwKVz9RhJEQi7tVCkduks1GUGS6',
'1128Nz11sqF4qNztppsp2rCdAUXRPzQka6WCnvPDvCJBnUNirYvw',
'112CJmutfGXyxfuvPiiMK4zrdc6jSus6tMvnQ9j4Tf9dSscm6SER',
'11UqADGT1B9B9oJsEiGQ6NvyiwtaogvaUUFgN9b3vaxTC6wMCyz',
'11q4gN7B6ovqhnwW3CaZVYuNVccWrx4qGTfLtRWPwFMFysx7HzD',
'11K31XKptX4gbk1LJ1oybErWxNuaZZPZHk2fdgpKofELU31Y4Gu',
'11y4f1y8vycYfEB2Mtp2tnihErs1raMXWTVRsSTjj7YWJiyEXPt',
'119i9qVaXU7C2YMW4Xkf9gLDCpWYtPiFnoAVAihB7CogtomvSqe',
'11gTMSvLKtKmDrymgJ1WoVfo6WVf1b7D5YK66iR47Ua1rxJ4fqb',
'11vdCcQhUw1jcZ6WdC8A3gxBi89SeSLWuSYK6AKoCridZ3ovqx2',
'112N4omQgTVcV1nmMSdj5rwA2VGyYU8DdM3WoJj7UKnyL1rM6j61',
'112ZLPTGX5GMipptERLTmfTdjhHhfkBQ7HaDeMYmr3xGU1Jphb68',
'11A2bkKUakAmnoQxxQrhpLY3fPFEu9ceNqMQyGH1gvXSnRA6Rnb',
'11e2ZGhWi8Y4ErdMfbCpaPTj6xPCeuxcZwZUha9RFSNEjMt222',
'11jkShSr3LYVQsPRJvCA9qzees7sbTjWkAyPMZ8oWqZt8v6mgfs',
'11st9TkPttnyXQZ1wttbA1CaDqn9ZczwJfdr7ZZSRqfN6am4tg2',
'1127pK5zDVwhn4nEtyn1UGJDtYKz5DoVXwoxgdURvB9sSB3EJci3',
'112QRUaj9WNAKjzkSmz8ReuVqHiuLSu2zKnZDgSzHtTrYceo7tNG',
'11251vfnmi1reEzrAh855sTWKz3m3NTwRcjkQy6T98bqd38rGfvo',
'113tq5UoVM1Ui7ecUABUSmjtmTHoD9jRhkB8wx69LESedCL1Ak7',
'11juiMhYRTG3BsFYSoewThVJZNwKfGSxA9M3V2K4CKtXmN9Gs2n',
'11TskfjCoFHqfV4FsA2PdnZ8EHgNTKybsAC4Mbad2L9yoQo1VM2',
'11M7bf7T8RAHpLiv7vvQLbXzaYrYMB2da5JwEd6fBbKWHCFRxr8',
'1127grbX1mznmHjP5QWw3X3GgshN2TqkbXNsR3XXG2hbqM65fRKB',
'11JPnMCBn7MJKH8ecopEdu9pB3boB7xwJYPaLJgeA9voq54wm8C',
'11yDjrTcSynpBWz1US4EBRv2XrGaJGBFresnzBbEUCAz8bEHUGT']

### Loading meatadata and reward data from Helium blockchain API

In [3]:
details_ll = []
reward_ll = []
w_ll = []

for address in tqdm(hbp_ids_list):
    details_ll.append(get_hotspot_details(address))
    reward_ll.append(get_hotspot_rewards(address, n_days=5))
    w_ll.append(list_witnesses_for_hotspot(address))
    
    


h_df = pd.DataFrame.from_records(details_ll)

100%|█████████████████████████████████████| 57/57 [02:39<00:00,  2.80s/it]


In [4]:
h_df['reward_5D'] = reward_ll

#### stats about witnesses - reward_scale of witnesses

In [5]:
reward_stats_df = pd.DataFrame.from_records([pd.DataFrame.from_records(x).reward_scale.describe().to_dict() if x else {'count':None, 'mean':None, 'std':None, 'min':None, '25%':None, '50%':None, '75%':None,'max':None} for x in w_ll ])

In [6]:
h_df= pd.concat([h_df, reward_stats_df], axis=1).copy()

#### HBP rewards vs their witnesses

In [7]:
h_df.sort_values('reward_5D', ascending=False)[['name','reward_scale', 'reward_5D', 'count', 'mean', 'std', 'min', '25%', '50%', '75%' ,'max']]

,name,reward_scale,reward_5D,count,mean,std,min,25%,50%,75%,max
8,elegant-tangerine-chameleon,0.599518,3.626865,59.0,0.577951,0.141151,0.199844,0.599518,0.599518,0.606064,1.000000
14,clumsy-daffodil-tuna,0.599518,3.251251,85.0,0.506050,0.180500,0.103653,0.310974,0.599518,0.601685,0.836014
5,creamy-ultraviolet-puma,1.000000,3.095379,128.0,0.882393,0.217810,0.333328,1.000000,1.000000,1.000000,1.000000
0,tangy-felt-caribou,1.000000,2.951633,79.0,0.910591,0.189857,0.333328,1.000000,1.000000,1.000000,1.000000
10,icy-tortilla-albatross,0.599518,2.793325,153.0,0.495353,0.232401,0.024872,0.299759,0.599518,0.606064,1.000000
12,straight-ocean-panther,1.000000,2.734745,21.0,0.577150,0.077305,0.299759,0.599518,0.599518,0.599518,0.609756
11,rich-golden-dog,0.575119,2.365156,160.0,0.769534,0.283060,0.087631,0.575119,1.000000,1.000000,1.000000
1,orbiting-tartan-sidewinder,1.000000,2.219880,159.0,0.689151,0.317048,0.086441,0.447052,0.727280,1.000000,1.000000
4,odd-mahogany-falcon,1.000000,2.058925,109.0,0.802461,0.244934,0.166672,0.558029,1.000000,1.000000,1.000000
2,joyous-olive-bull,1.000000,1.753667,132.0,0.663432,0.337351,0.087631,0.333328,0.615387,1.000000,1.000000


In [8]:
h_df.sort_values('reward_5D', ascending=False)[['name','reward_scale', 'reward_5D', 'count', 'mean', 'std', 'min', '25%', '50%', '75%' ,'max']].to_csv('reward_vs_witnesses_analysis.csv')

In [9]:
h_df= pd.read_csv('reward_vs_witnesses_analysis.csv')

In [10]:
for scale in ['mean', '25%', '50%', '75%']:
    h_df[f'count_x_{scale}'] = h_df['count'] * h_df[scale]

In [11]:
h_df[['reward_scale', 'reward_5D', 'count', 'mean', '25%', '50%', '75%', 'max',  'count_x_mean', 'count_x_25%', 'count_x_50%', 'count_x_75%']].corr()

,reward_scale,reward_5D,count,mean,25%,50%,75%,max,count_x_mean,count_x_25%,count_x_50%,count_x_75%
reward_scale,1.000000,0.506252,-0.355233,0.684886,0.580460,0.674449,0.617274,-0.224621,0.164369,0.247623,0.291180,0.186037
reward_5D,0.506252,1.000000,-0.241424,0.499129,0.523114,0.572395,0.335994,-0.352526,0.144670,0.273702,0.317015,0.083107
count,-0.355233,-0.241424,1.000000,-0.225438,-0.290214,-0.322170,-0.069381,0.401127,0.719684,0.484407,0.560553,0.681859
mean,0.684886,0.499129,-0.225438,1.000000,0.886583,0.922607,0.892204,0.042129,0.491549,0.639444,0.593489,0.455865
25%,0.580460,0.523114,-0.290214,0.886583,1.000000,0.820371,0.667063,-0.029849,0.348092,0.658269,0.452293,0.255799
50%,0.674449,0.572395,-0.322170,0.922607,0.820371,1.000000,0.776608,-0.022099,0.353711,0.518890,0.573247,0.299908
75%,0.617274,0.335994,-0.069381,0.892204,0.667063,0.776608,1.000000,0.104779,0.572887,0.587579,0.613696,0.658660
max,-0.224621,-0.352526,0.401127,0.042129,-0.029849,-0.022099,0.104779,1.000000,0.355986,0.284719,0.295204,0.318541
count_x_mean,0.164369,0.144670,0.719684,0.491549,0.348092,0.353711,0.572887,0.355986,1.000000,0.893459,0.935860,0.951239
count_x_25%,0.247623,0.273702,0.484407,0.639444,0.658269,0.518890,0.587579,0.284719,0.893459,1.000000,0.881734,0.784224


In [12]:
details_df = pd.DataFrame(details_ll)

details_df.dropna(subset=['location_hex'], inplace=True)

In [13]:
def list_hotsopts_within_location_hex(location_hex: str):
    
    url = 'https://api.helium.io/v1/hotspots/hex/' + location_hex
    r = requests.get(url)
    hex_hs = r.json()['data']
    hex_hs_df = pd.DataFrame(hex_hs)
    
    return hex_hs_df

### Analysing the reward scale of other hotspots within the same hex

In [15]:
hex_reward_stats = []
hex_elevation_stats = []

for i, row in details_df.iterrows():
    
    location_hex = row['location_hex']
    name = row['name'].replace('-', " ")
    hex_hs_df = list_hotsopts_within_location_hex(location_hex)
    hex_hs_df=hex_hs_df[hex_hs_df.status.apply(lambda x:x['online'] == 'online')].copy()

    hex_hs_df.dropna(subset=['reward_scale'])
    hex_hs_df = list_hotsopts_within_location_hex(location_hex)
    hex_reward_stats.append(hex_hs_df.reward_scale.tolist())
    hex_elevation_stats.append(hex_hs_df.elevation.tolist())
        

In [16]:
details_df['hex_rewards'] = hex_reward_stats
details_df['hex_elevations'] = hex_elevation_stats

In [17]:
details_df['hex_mean'] = details_df['hex_rewards'].apply(lambda x: np.mean([i for i in x if i]))

In [18]:
details_df['count'] = details_df['hex_rewards'].apply(lambda x: len([i for i in x if i]))

In [19]:
details_df.sort_values('reward_scale', ascending=False, inplace=True)

In [20]:
details_df[['name','reward_scale', 'hex_mean', 'hex_rewards', 'hex_elevations', 'count']].tail(20)

,name,reward_scale,hex_mean,hex_rewards,hex_elevations,count
22,sneaky-khaki-reindeer,0.405182,0.538935,"[0.8077545166015625, 0.405181884765625, 0.4038...","[0, 58, 40]",3
16,salty-topaz-elk,0.386688,0.386688,[0.386688232421875],[65],1
7,flaky-tartan-coyote,0.334381,0.193146,"[0.16436767578125, 0.334381103515625, 0.164001...","[0, 20, 9, 2]",4
47,attractive-fern-condor,0.323242,0.323242,[0.3232421875],[39],1
38,funny-jade-skunk,0.321213,0.343633,"[0.3200836181640625, 0.3212127685546875, 0.389...","[2, 67, 0]",3
21,flat-powder-pelican,0.320648,0.322271,"[0.3255157470703125, 0.320648193359375, 0.3206...","[0, 80, 0]",3
13,fresh-coffee-terrier,0.310135,0.321350,"[0.3105010986328125, 0.3101348876953125, 0.343...","[13, 15, 4]",3
43,cheery-orchid-gazelle,0.307861,0.308873,"[0.307861328125, 0.3081817626953125, 0.3105773...","[42, 10, 0]",3
24,future-carmine-penguin,0.284210,0.279648,"[0.2840118408203125, 0.284210205078125, 0.2840...","[25, 0, 10, 0, 0, 0, 0, 0]",8
44,prehistoric-saffron-otter,0.281326,0.299611,"[0.278778076171875, 0.2813262939453125, 0.3387...","[1, 40, 0]",3


In [21]:
details_df[['reward_scale', 'hex_mean', 'count']].corr()

,reward_scale,hex_mean,count
reward_scale,1.000000,0.969755,-0.512458
hex_mean,0.969755,1.000000,-0.496035
count,-0.512458,-0.496035,1.000000
